## 1. Packages

In [ ]:
using Agents
using GLMakie
using Observables
using Random
using ColorSchemes

## 2. Agent definition

In [24]:
# Define Agent Type
@agent struct OP(GridAgent{2})
    opinion::Float64
end

## 3. Model initialization 

In [ ]:
# Initalize Model
function initialize(; M=10,
    local_int::Bool=false,
    learning_rate::Float64=0.5,
    consensus_threshold::Float64=0.05,
    interaction_mode::Symbol=:positive,
    confidence_threshold::Float64=0.10)

    space = GridSpaceSingle((M, M); periodic=true, metric=:manhattan)

    props = Dict(
        :local_int => local_int,
        :learning_rate => learning_rate,
        :consensus_threshold => consensus_threshold,
        :interaction_mode => interaction_mode,
        :confidence_threshold => confidence_threshold,
        :consensus => false,
        :time_to_consensus => 0,
    )

    model = StandardABM(
        OP, space;
        properties=props,
        model_step!,
        scheduler=Schedulers.Randomly(),
    )

    for _ in 1:(M^2) # Fill grid completely
        add_agent_single!(model; opinion=2rand(abmrng(model)) - 1) # Opinion in [-1, 1]
    end
    return model
end

initialize (generic function with 1 method)

## 4. Helper - sample two distinct agents

In [ ]:
# Get Two Distinct Agent IDs 
@inline function two_distinct_ids(n::Int)
    i = rand(1:n)
    j = rand(1:(n-1))
    j >= i && (j += 1)
    return i, j
end

two_distinct_ids (generic function with 1 method)

## 5. Interaction mechanism 

In [ ]:
# Interaction between two agents based on the selected mechanism
function interact!(a::OP, b::OP, model)
    x1, x2 = a.opinion, b.opinion # Current opinions
    lr = model.learning_rate
    d = model.confidence_threshold
    mode = model.interaction_mode
    Δ = abs(x1 - x2) # Opinion difference 

    if mode === :positive # Standard positve interaction
        x1n = x1 + lr * (x2 - x1)
        x2n = x2 + lr * (x1 - x2)

    elseif mode === :bounded # Bounded confidence
        Δ < d || return
        x1n = x1 + lr * (x2 - x1)
        x2n = x2 + lr * (x1 - x2)

    elseif mode === :negative # Bounded confidence with negative interaction
        if Δ < d
            x1n = x1 + lr * (x2 - x1)
            x2n = x2 + lr * (x1 - x2)

        elseif Δ > d
            if x1 > x2
                x1n = x1 + lr * (x1 - x2) * (1 - x1) * 0.5
                x2n = x2 + lr * (x2 - x1) * (1 + x2) * 0.5
            else
                x1n = x1 + lr * (x1 - x2) * (1 + x1) * 0.5
                x2n = x2 + lr * (x2 - x1) * (1 - x2) * 0.5
            end
        else
            # Δ == d: no change
            return
        end
    end


    a.opinion = x1n
    b.opinion = x2n
end


interact! (generic function with 1 method)

## 6. Helper - clique detection - metrics

In [ ]:
# Clique detection and counting functions
function clique_ranges(model)::Vector{Float64}
    d = model.confidence_threshold
    xs = sort!([ag.opinion for ag in allagents(model)]) # Sorted opinions to find ranges
    n = length(xs)
    n == 0 && return Float64[]
    n == 1 && return [0.0]

    ranges = Float64[]
    cmin = xs[1]
    cmax = xs[1]
    prev = xs[1]

    # Iterate through sorted opinions and find ranges
    @inbounds for k in 2:n
        x = xs[k]
        if (x - prev) > d
            push!(ranges, cmax - cmin)
            cmin = x
            cmax = x
        else
            cmax = x
        end
        prev = x
    end
    push!(ranges, cmax - cmin)
    return ranges
end

clique_ranges (generic function with 1 method)

## 7. Dynamics - model_step!

In [ ]:
function model_step!(model)
    n = nagents(model)
    n <= 1 && return

    # Select two agents to interact (global or local-neighborhood)
    if model.local_int
        i = rand(1:n)
        a = model[i]
        j = random_nearby_id(a, model, 1)
        b = model[j]
    else
        i, j = two_distinct_ids(n)
        a = model[i]
        b = model[j]
    end

    interact!(a, b, model)

    # Consensus check: all clique opinion ranges below consensus_threshold
    ranges = clique_ranges(model)
    if !isempty(ranges) && all(r -> r < model.consensus_threshold, ranges) && !model.consensus
        model.time_to_consensus = abmtime(model)
        model.consensus = true
    end
end


model_step! (generic function with 1 method)

## 8. GUI setup 

In [ ]:
# Visualization setup

opinion_cmap = cgrad(:RdBu, rev=true)  # blue at -1, red at +1

# push slider values away from the extremes by "pad"
remap_t(t; pad=0.12) = pad + (1 - 2pad) * clamp(t, 0, 1)

agent_color(a::OP) = get(opinion_cmap, remap_t((a.opinion + 1) / 2; pad=0.15))

# Custom GUI + dynamic plots
GLMakie.activate!()

init_base = (M=10, local_int=false)
init_rate = 0.5
init_thr = 0.10
init_cons = 0.05
init_mode = :positive

model = initialize(; init_base..., learning_rate=init_rate,
    consensus_threshold=init_cons,
    confidence_threshold=init_thr, interaction_mode=init_mode)

fig = Figure(size=(1350, 740))

# Layout
ax_grid = fig[1, 1] = Axis(fig; title="Agents on grid")
ax_lines = fig[1, 2] = Axis(fig; title="All opinions over time",
    xlabel="step", ylabel="opinion")
controls = fig[1, 3] = GridLayout()

# controls layout
colsize!(fig.layout, 3, Fixed(280))               # sidebar width
colgap!(fig.layout, 20);
rowgap!(fig.layout, 10);  # spacing between columns/rows

# ABM view
abmobs = abmplot!(ax_grid, model;
    agent_color=agent_color,
    agent_marker=:rect,
    agent_size=30,
    add_controls=false,
    enable_inspection=false)

# spaghetti plot
limits!(ax_lines, (nothing, nothing), (-1, 1))
traces = IdDict{Int,Observable{Vector{Point2f}}}()

function rebuild_spaghetti!(m)
    empty!(traces)
    empty!(ax_lines)
    t0 = Float32(abmtime(m))
    for ag in allagents(m)
        obs = Observable([Point2f(t0, Float32(ag.opinion))])
        lines!(ax_lines, obs; linewidth=1, transparency=true)
        traces[ag.id] = obs
    end
    autolimits!(ax_lines)
end
rebuild_spaghetti!(abmobs.model[])

last_step = Ref(Int(abmtime(abmobs.model[])))
on(abmobs.model) do m
    t = Float32(abmtime(m))
    if Int(t) <= last_step[]           # reset check
        rebuild_spaghetti!(m)
    else
        for ag in allagents(m)
            obs = traces[ag.id]
            buf = obs[]
            push!(buf, Point2f(t, Float32(ag.opinion)))
            obs[] = buf
        end
        reset_limits!(ax_lines)        # recompute X; keep Y fixed
    end
    last_step[] = Int(t)
end

# Sidebar controls
# Row 1: buttons
btn_row = controls[1, 1] = GridLayout()
run_btn = Button(btn_row[1, 1], label="Run")
step_btn = Button(btn_row[1, 2], label="Step")
reset_btn = Button(btn_row[1, 3], label="Reset")

# Row 2: sliders (SPU (steps per update) & Sleep (speed))
Label(controls[2, 1], "Simulation", font=:bold, halign=:left)
sg_sim = SliderGrid(
    controls[3, 1],
    (label="SPU", range=1:50, format="{:.0f}", startvalue=1),
    (label="Sleep (s)", range=0.0:0.01:0.3, format="{:.02f}s", startvalue=0.00),
    width=260, tellheight=false
)

spu_slider = sg_sim.sliders[1]
sleep_slider = sg_sim.sliders[2]

# Row 3: model params (Menu + slidergrid)
Label(controls[4, 1], "Model parameters", font=:bold, halign=:left)

Label(controls[5, 1], "Mode", halign=:left)
mode_menu = Menu(controls[6, 1],
    options=[("Positive", :positive),
        ("Bounded", :bounded),
        ("Bounded+Negative", :negative)],
    width=260)
mode_menu.selection[] = init_mode

# Locality control, label + toggle
local_row = GridLayout()
local_row[1, 1] = Label(fig, "Local interactions"; halign=:left)
local_toggle = Toggle(fig; active=init_base.local_int)
local_row[1, 2] = local_toggle
colgap!(local_row, 12)
colsize!(local_row, 1, Auto())
colsize!(local_row, 2, Fixed(50))

controls[7, 1] = local_row


# Sliders below locality
sg_model = SliderGrid(
    controls[8, 1],
    (label="Learning rate", range=0.0:0.01:1.0, format="{:.02f}", startvalue=init_rate),
    (label="Confidence threshold", range=0.0:0.01:1.0, format="{:.02f}", startvalue=init_thr),
    (label="Consensus threshold", range=0.0:0.01:1.0, format="{:.02f}", startvalue=init_cons),
    width=260, tellheight=false
)

lr_slider = sg_model.sliders[1]
thr_slider = sg_model.sliders[2]
cons_slider = sg_model.sliders[3]

# adjust gaps inside the sidebar
colgap!(controls, 0);
rowgap!(controls, 8);

# controls
running = Observable(false)
on(run_btn.clicks) do _
    running[] = !running[]
    run_btn.label[] = running[] ? "Pause" : "Run"
end

# Step button
@async begin
    while true
        if running[]
            Agents.step!(abmobs, Int(round(spu_slider.value[])))

            # Stop automatically once consensus is reached
            if getproperty(abmobs.model[], :consensus) === true
                running[] = false
                run_btn.label[] = "Run"
            end

            sleep(sleep_slider.value[])
        else
            sleep(0.05)
        end
    end
end


# Reset button
on(reset_btn.clicks) do _
    abmobs.model[] = initialize(; M=init_base.M,
        local_int=local_toggle.active[],
        learning_rate=Float64(lr_slider.value[]),
        consensus_threshold=Float64(cons_slider.value[]),
        confidence_threshold=Float64(thr_slider.value[]),
        interaction_mode=mode_menu.selection[])
    running[] = false
    run_btn.label[] = "Run"
end

# update params live when adjusted
on(mode_menu.selection) do sym
    abmobs.model[].interaction_mode = sym
end

# locality toggle
on(local_toggle.active) do v
    abmobs.model[].local_int = v
end
# sliders
on(lr_slider.value) do v
    abmobs.model[].learning_rate = Float64(v)
end
# sliders
on(thr_slider.value) do v
    abmobs.model[].confidence_threshold = Float64(v)
end
# sliders
on(cons_slider.value) do v
    abmobs.model[].consensus_threshold = Float64(v)
end




ObserverFunction defined at In[30]:190 operating on Observable{Any}(0.05)

## 9. Launch GUI

In [31]:
# Run to open the GUI
display(fig)

GLMakie.Screen(...)